In [2]:
rawData=sc.textFile("/home/stan/Downloads/train_noheader.tsv")

In [3]:
records=rawData.map(lambda line: line.split("\t"))

In [4]:
records.first()

[u'"http://www.bloomberg.com/news/2010-12-23/ibm-predicts-holographic-calls-air-breathing-batteries-by-2015.html"',
 u'"4042"',
 u'"{""title"":""IBM Sees Holographic Calls Air Breathing Batteries ibm sees holographic calls, air-breathing batteries"",""body"":""A sign stands outside the International Business Machines Corp IBM Almaden Research Center campus in San Jose California Photographer Tony Avelar Bloomberg Buildings stand at the International Business Machines Corp IBM Almaden Research Center campus in the Santa Teresa Hills of San Jose California Photographer Tony Avelar Bloomberg By 2015 your mobile phone will project a 3 D image of anyone who calls and your laptop will be powered by kinetic energy At least that s what International Business Machines Corp sees in its crystal ball The predictions are part of an annual tradition for the Armonk New York based company which surveys its 3 000 researchers to find five ideas expected to take root in the next five years IBM the world 

In [5]:
from pyspark.mllib.regression import LabeledPoint
from pyspark.mllib.linalg import Vectors

In [6]:
def convert (x):
    if x=="?":
        return 0.0
    else:
        return float(x)

In [7]:
def createLabeledPoint(r):
    trimmed=map(lambda x: x.replace("\"",""),r)
    label=int(trimmed[len(r)-1])
    features=map(lambda x: convert(x),trimmed[4:len(r)-1])
    return LabeledPoint(label, Vectors.dense(features))

In [8]:
createLabeledPoint(records.first())


LabeledPoint(0.0, [0.789131,2.055555556,0.676470588,0.205882353,0.047058824,0.023529412,0.443783175,0.0,0.0,0.09077381,0.0,0.245831182,0.003883495,1.0,1.0,24.0,0.0,5424.0,170.0,8.0,0.152941176,0.079129575])

In [9]:
data=records.map(lambda r: createLabeledPoint(r))

In [10]:
data.cache()

PythonRDD[4] at RDD at PythonRDD.scala:43

In [11]:
data.count()

7395

In [12]:
def positiveConvert (x):
    if x=="?":
        return 0.0
    else:
        tmp=float(x)
        if tmp < 0.0:
            return 0.0
        else:
            return tmp
        

In [13]:
def createPositiveLabeledPoint(r):
    trimmed=map(lambda x: x.replace("\"",""),r)
    label=int(trimmed[len(r)-1])
    features=map(lambda x: positiveConvert(x),trimmed[4:len(r)-1])
    return LabeledPoint(label, Vectors.dense(features))

In [14]:
nbData=records.map(lambda x: createPositiveLabeledPoint(x))

In [15]:
nbData.first()

LabeledPoint(0.0, [0.789131,2.055555556,0.676470588,0.205882353,0.047058824,0.023529412,0.443783175,0.0,0.0,0.09077381,0.0,0.245831182,0.003883495,1.0,1.0,24.0,0.0,5424.0,170.0,8.0,0.152941176,0.079129575])

In [16]:
nbData.count()

7395

In [17]:
from pyspark.mllib.classification import LogisticRegressionWithSGD
from pyspark.mllib.classification import SVMWithSGD
from pyspark.mllib.classification import NaiveBayes
from pyspark.mllib.tree import DecisionTree


In [19]:
numIterations=10
maxTreeDepth =5

In [20]:
lrModel= LogisticRegressionWithSGD.train(data,numIterations)

In [21]:
svmModel = SVMWithSGD.train(data,numIterations)

In [22]:
nbModel = NaiveBayes.train(nbData)

In [23]:
dtModel=DecisionTree.trainClassifier(data,2,{})

# Use logisticRegression to classify

In [24]:
dataPoint = data.first()
prediction=lrModel.predict(dataPoint.features)

In [25]:
prediction

1

In [26]:
trueLabel = dataPoint.label

In [27]:
trueLabel

0.0

In [28]:
predictions = lrModel.predict(data.map(lambda x: x.features))

In [29]:
predictions.take(5)

[1, 1, 1, 1, 1]

## Accuracy and prediction error

In [36]:
from operator import add

In [37]:
lrTotalCorrect=data.map(lambda x: lrModel.predict(x.features)==x.label).reduce(add)

In [38]:
lrTotalCorrect

3806

In [46]:
lrAccuracy=lrTotalCorrect*1.0/data.count()

In [47]:
lrAccuracy

0.5146720757268425

In [43]:
3806.0/7395

0.5146720757268425

In [49]:
svmTotalCorrect=data.map(lambda x: svmModel.predict(x.features)==x.label).reduce(add)
nbTotalCorrect=nbData.map(lambda x: nbModel.predict(x.features)==x.label).reduce(add)

In [55]:
dtModel.predict(data.first().features)

0.0

In [64]:
dtTotalCorrect=data.map(lambda x: x.features)

In [66]:
dtModel

DecisionTreeModel classifier of depth 5 with 61 nodes

In [67]:
dtModel.predict(data.first().features)

0.0

In [115]:
dtTotalCorrect=sum(map(lambda x: dtModel.predict(x.features),data.collect()))

In [116]:
dtTotalCorrect

3549.0

In [73]:
svmAccuracy=svmTotalCorrect*1.0 / data.count()

In [74]:
svmAccuracy

0.5146720757268425

In [80]:
nbTotalCorrect

True

In [78]:
nbAccuracy

0.00013522650439486139

In [102]:
nbTotalCorrect=nbData.map(lambda x: nbModel.predict(x.features)==x.label).collect()

In [111]:
nbAccuracy=sum(nbTotalCorrect)*1.0/nbData.count()

In [112]:
nbAccuracy

0.58039215686274515

In [117]:
dtAccuracy=dtTotalCorrect/data.count()

In [118]:
dtAccuracy

0.47991886409736306

##Precision and recall